### IMPORTS

In [42]:
import json
import hashlib
import pkg_resources.py2_warn

In [2]:
from web3 import Web3

In [3]:
import subprocess
import sys

In [4]:
print('[X] Booting up...')

[X] Booting up...


### LOAD DATA FROM EXTERNAL FILES

In [5]:
with open('config/settings.json') as json_file:
    settings = json.load(json_file)

In [6]:
with open('config/identifier.json') as json_file:
    device_info = json.load(json_file)

In [7]:
with open('config/latest.json') as json_file:
    latest = json.load(json_file)

### IPFS & BLOCKCHAIN GATEWAY CONNECTIONS

In [8]:
class gateway:
    def __init__(self, foo):
        self.host = foo['host']
        self.port = foo['port']

In [9]:
blockchain = gateway(settings['gateways']['blockchain'])

In [10]:
print('[X] Gateway set!')

[X] Gateway set!


### BLOCKCHAIN PUBLIC/PRIVATE KEYS

In [11]:
class keypair:
    def __init__(self, foo):
        self.public = foo['public']
        self.private = foo['private']

In [12]:
keys = keypair(settings['keys'])

In [13]:
print('[X] Keys set!')

[X] Keys set!


### HASH DEVICE INFO FOR IDENTIFIER

In [14]:
def hash_id(data):
    
    # REMOVE WHITESPACES
    to_string = json.dumps(data, sort_keys=False, indent=2)
    
    # ENCODE THE STRING WITH UTF8
    encoded = to_string.encode("utf-8")
    
    # HASH ENCODED DATA
    hashed = hashlib.sha256(encoded).hexdigest()
    
    return hashed

In [15]:
class create_device():
    def __init__(self):
        self.name = device_info['Name']
        self.hash = hash_id(device_info)
    
    # LOCATE & SET DEVICE CONTRACT ADDRESS
    def set_contract(self, _contract):
        self.contract = _contract
    
    # REDIRECT TO SMART CONTRACT INTERFACE
    def read(self, details):
        return self.contract.read(details)
    
    # REDIRECT TO SMART CONTRACT INTERFACE
    def write(self, details):
        return self.contract.write(details)
    
    # REDIRECT TO SMART CONTRACT INTERFACE
    def event(self, name):
        return self.contract.event(name)

In [16]:
device = create_device()

In [17]:
print('[X] Device identifier hashed!')

[X] Device identifier hashed!


### CONNECT TO CHAIN VIA WEBSOCKET

In [18]:
web3 = Web3(Web3.WebsocketProvider('ws://' + blockchain.host + ':' + blockchain.port))

In [19]:
if web3.isConnected():
    print('[X] Connected with gateway!')
else:
    print('[ ] Could not connect to gateway!')
    exit

[X] Connected with gateway!


### CONSTRUCT SMART CONTRACT INTERFACE

In [20]:
class contract:
    
    # ON LOAD..
    def __init__(self, block):
        
        # CONSTRUCT USABLE CONTRACT
        self.contract = web3.eth.contract(
            address = block['address'],
            abi = block['abi']
        )
        
        # SET ADDRESS REFERENCE
        self.address = block['address']
    
    # READ FROM CONTRACT
    def read(self, details):
        
        # WITH PARAMS
        if ('params' in details):
            return self.contract.functions[details['func']](details['params']).call()
        
        # WITHOUT PARAMS
        else:
            return self.contract.functions[details]().call()
    
    # WRITE TO CONTRACT
    def write(self, details):
        try:
            
            # CREATE BASE TRANSACTION
            tx = {
                'from': keys.public,
                'to': self.contract.address,
                'data': self.contract.encodeABI(
                    fn_name = details['func'],
                    args = details['params']
                )
            }
            
            # ESTIMATE GAS VALUE & STITCH IN REMAINING PROPS
            tx['gas'] = web3.eth.estimateGas(tx)
            tx['gasPrice'] = web3.toWei(20, 'gwei')
            tx['nonce'] = web3.eth.getTransactionCount(keys.public)

            # SIGN TRANSCTION WITH PRIVATE KEY
            signed = web3.eth.account.sign_transaction(tx,
                private_key = keys.private
            )

            # SEND THE TRANSACTION
            web3.eth.sendRawTransaction(signed.rawTransaction)

            # SUCCESS
            return True
        
        # IF THE TRANSACTION IS REVERTED, SHOW ERROR
        except ValueError as error:
            return error
    
    # EVENT FILTER
    def event(self, name):
        return self.contract.events[name].createFilter(fromBlock="latest")

### STITCH TOGETHER MANAGER CONTRACTS

In [21]:
user_manager = contract(latest['usermanager'])

In [22]:
device_manager = contract(latest['devicemanager'])

In [23]:
task_manager = contract(latest['taskmanager'])

In [24]:
token_manager = contract(latest['tokenmanager'])

In [25]:
print('[X] Manager contracts set!')

[X] Manager contracts set!


### CONTRUCT DEVICE CONTRACT INTERFACE

In [26]:
temp_contract = contract({
    'address': device_manager.read({
        'func': 'fetch_device',
        'params': device.hash
    }),
    'abi': latest['device']['abi']
})

In [27]:
device.set_contract(temp_contract)

In [28]:
print('[X] Device interface set!')

[X] Device interface set!


### SUBSCRIBE TO UPDATE EVENTS

In [29]:
print('[X] Listening for events...\n')

[X] Listening for events...



In [35]:
update_event = device.event('middleware')

In [36]:
config_event = device.event('config')

In [41]:
while(True):
    
    # UPDATE MIDDLEWARE EVENT
    for event in update_event.get_new_entries():
    
        # PRINT REACTION
        print('Update event triggered!')
        
        # TRIGGER UPDATE SCRIPT
        # subprocess.call('./patcher')
              
        # CLOSE LANCHER
        # sys.exit(0)

Update event triggered!
Config event triggered!


KeyboardInterrupt: 